In [ ]:
from collections import deque
import time

class EdmondsKarp:
    def __init__(self, n):
        """初始化图结构，n为节点总数"""
        self.n = n
        self.graph = [[0] * n for _ in range(n)]  # 邻接矩阵存储图
    
    def add_edge(self, fr, to, cap):
        """添加边，fr为起点，to为终点，cap为容量"""
        self.graph[fr][to] = cap
    
    def bfs(self, source, sink, parent):
        """BFS寻找从源点到汇点的增广路径，返回是否找到"""
        visited = [False] * self.n
        queue = deque([source])
        visited[source] = True
        parent[:] = [-1] * self.n  # 记录路径
        
        while queue:
            u = queue.popleft()
            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    queue.append(v)
                    parent[v] = u
                    visited[v] = True
                    if v == sink:
                        return True
        return False
    
    def max_flow(self, source, sink):
        """计算从源点到汇点的最大流"""
        parent = [-1] * self.n
        max_flow = 0
        
        while self.bfs(source, sink, parent):
            # 找到增广路径上的最小残留容量
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.graph[u][v])
                v = u
            
            # 更新最大流
            max_flow += path_flow
            
            # 更新残留网络（添加反向边）
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减少流量
                self.graph[v][u] += path_flow  # 反向边增加流量
                v = u
        
        return max_flow

def is_eliminated(teams, games, team_idx):
    """判断球队是否被淘汰，返回(是否淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    ek = EdmondsKarp(total_nodes)
    
    # 1. 源点→比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                ek.add_edge(s, game_idx, games[i][j])                # 源点到比赛节点
                ek.add_edge(game_idx, game_nodes + i + 1, 10**9)     # 比赛到球队i
                ek.add_edge(game_idx, game_nodes + j + 1, 10**9)     # 比赛到球队j
                game_idx += 1
    
    # 2. 球队节点→汇点（设置容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 目标球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            ek.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量为理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)          # 其他球队最多赢到max_win_x场
            ek.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数
    
    # 3. 计算最大流与总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = ek.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 最大流不足则被淘汰
    return is_elim, max_win_x

# 题目数据集
team_names = ["Atlanta", "Philly", "New York", "Montreal"]
teams = [
    (83, 8),   # Atlanta：已胜83，剩余8场 → 理论最大91
    (80, 3),   # Philly：已胜80，剩余3场 → 理论最大83
    (78, 6),   # New York：已胜78，剩余6场 → 理论最大84
    (77, 3)    # Montreal：已胜77，剩余3场 → 理论最大80
]
games = [
    [0, 1, 6, 1],  # Atlanta与Philly(1)、NY(6)、Montreal(1)
    [0, 0, 0, 2],  # Philly与NY(0)、Montreal(2)
    [0, 0, 0, 0],  # NY与Montreal(0)
    [0, 0, 0, 0]   # Montreal无剩余比赛
]

# 主程序：计时并输出结果
start_time = time.time()
for i in range(4):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

In [ ]:
EdmondsKarp(G, s, t):
    // 初始化所有边的流量为0
    for each edge (u, v) in G.edges:
        (u, v).flow = 0
    
    max_flow = 0
    // 存储增广路径中每个节点的前驱节点
    parent = array of size G.vertices, initialized to -1
    
    while BFS(G, s, t, parent):
        // 找到增广路径上的最小残留容量
        path_flow = infinity
        v = t
        while v != s:
            u = parent[v]
            path_flow = min(path_flow, G.residual_capacity(u, v))
            v = u
        
        // 增加流量
        max_flow += path_flow
        
        // 更新残留网络
        v = t
        while v != s:
            u = parent[v]
            G.add_flow(u, v, path_flow)
            v = u
    
    return max_flow


BFS(G, s, t, parent):
    // 使用BFS寻找从s到t的增广路径
    visited = array of size G.vertices, initialized to false
    queue = deque()
    queue.append(s)
    visited[s] = true
    parent[s] = -1
    
    while queue not empty:
        u = queue.popleft()
        for each v in G.adjacent(u):
            if not visited[v] and G.residual_capacity(u, v) > 0:
                queue.append(v)
                parent[v] = u
                visited[v] = true
                if v == t:
                    return true
    
    return false

In [ ]:
from collections import deque
import time

class EdmondsKarp:
    def __init__(self, n):
        """初始化图结构，n为节点总数"""
        self.n = n
        self.graph = [[0] * n for _ in range(n)]  # 邻接矩阵存储图
    
    def add_edge(self, fr, to, cap):
        """添加边，fr为起点，to为终点，cap为容量"""
        self.graph[fr][to] = cap
    
    def bfs(self, source, sink, parent):
        """BFS寻找从源点到汇点的增广路径，返回是否找到"""
        visited = [False] * self.n
        queue = deque([source])
        visited[source] = True
        parent[:] = [-1] * self.n  # 记录路径
        
        while queue:
            u = queue.popleft()
            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    queue.append(v)
                    parent[v] = u
                    visited[v] = True
                    if v == sink:
                        return True
        return False
    
    def max_flow(self, source, sink):
        """计算从源点到汇点的最大流"""
        parent = [-1] * self.n
        max_flow = 0
        
        while self.bfs(source, sink, parent):
            # 找到增广路径上的最小残留容量
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.graph[u][v])
                v = u
            
            # 更新最大流
            max_flow += path_flow
            
            # 更新残留网络（添加反向边）
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减少流量
                self.graph[v][u] += path_flow  # 反向边增加流量
                v = u
        
        return max_flow

def is_eliminated(teams, games, team_idx):
    """判断球队是否被淘汰，返回(是否淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    ek = EdmondsKarp(total_nodes)
    
    # 1. 源点→比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                ek.add_edge(s, game_idx, games[i][j])                # 源点到比赛节点
                ek.add_edge(game_idx, game_nodes + i + 1, 10**9)     # 比赛到球队i
                ek.add_edge(game_idx, game_nodes + j + 1, 10**9)     # 比赛到球队j
                game_idx += 1
    
    # 2. 球队节点→汇点（设置容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 目标球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            ek.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量为理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)          # 其他球队最多赢到max_win_x场
            ek.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数
    
    # 3. 计算最大流与总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = ek.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 最大流不足则被淘汰
    return is_elim, max_win_x

# 解析新数据集
team_names = ["New_York", "Baltimore", "Boston", "Toronto", "Detroit"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (75, 28),   # New_York：已胜75，剩余28场 → 理论最大103
    (71, 28),   # Baltimore：已胜71，剩余28场 → 理论最大99
    (69, 27),   # Boston：已胜69，剩余27场 → 理论最大96
    (63, 27),   # Toronto：已胜63，剩余27场 → 理论最大90
    (49, 27)    # Detroit：已胜49，剩余27场 → 理论最大76
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
# 从数据中解析：
# New_York与其他队：0 3 8 7 3 → 与Baltimore(3)、Boston(8)、Toronto(7)、Detroit(3)
# Baltimore与其他队：3 0 2 7 7 → 与Boston(2)、Toronto(7)、Detroit(7)
# Boston与其他队：8 2 0 0 3 → 与Toronto(0)、Detroit(3)
# Toronto与其他队：7 7 0 0 3 → 与Detroit(3)
# Detroit与其他队：3 7 3 3 0 → 无（i<j）
games = [[0 for _ in range(5)] for _ in range(5)]
# 填充上三角矩阵
games[0][1] = 3  # New_York vs Baltimore
games[0][2] = 8  # New_York vs Boston
games[0][3] = 7  # New_York vs Toronto
games[0][4] = 3  # New_York vs Detroit
games[1][2] = 2  # Baltimore vs Boston
games[1][3] = 7  # Baltimore vs Toronto
games[1][4] = 7  # Baltimore vs Detroit
games[2][3] = 0  # Boston vs Toronto
games[2][4] = 3  # Boston vs Detroit
games[3][4] = 3  # Toronto vs Detroit

# 确保矩阵对称性（下三角等于上三角）
for i in range(5):
    for j in range(i+1, 5):
        games[j][i] = games[i][j]

# 主程序：逐个判断球队 + 统计运行时间
start_time = time.time()
for i in range(5):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

In [9]:
from collections import deque
import time

class EdmondsKarp:
    def __init__(self, n):
        """初始化图结构，n为节点总数"""
        self.n = n
        self.graph = [[0] * n for _ in range(n)]  # 邻接矩阵存储图
    
    def add_edge(self, fr, to, cap):
        """添加边，fr为起点，to为终点，cap为容量"""
        self.graph[fr][to] = cap
    
    def bfs(self, source, sink, parent):
        """BFS寻找从源点到汇点的增广路径，返回是否找到"""
        visited = [False] * self.n
        queue = deque([source])
        visited[source] = True
        parent[:] = [-1] * self.n  # 记录路径
        
        while queue:
            u = queue.popleft()
            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    queue.append(v)
                    parent[v] = u
                    visited[v] = True
                    if v == sink:
                        return True
        return False
    
    def max_flow(self, source, sink):
        """计算从源点到汇点的最大流"""
        parent = [-1] * self.n
        max_flow = 0
        
        while self.bfs(source, sink, parent):
            # 找到增广路径上的最小残留容量
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.graph[u][v])
                v = u
            
            # 更新最大流
            max_flow += path_flow
            
            # 更新残留网络（添加反向边）
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减少流量
                self.graph[v][u] += path_flow  # 反向边增加流量
                v = u
        
        return max_flow

def is_eliminated(teams, games, team_idx):
    """判断球队是否被淘汰，返回(是否淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    ek = EdmondsKarp(total_nodes)
    
    # 1. 源点→比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                ek.add_edge(s, game_idx, games[i][j])                # 源点到比赛节点
                ek.add_edge(game_idx, game_nodes + i + 1, 10**9)     # 比赛到球队i
                ek.add_edge(game_idx, game_nodes + j + 1, 10**9)     # 比赛到球队j
                game_idx += 1
    
    # 2. 球队节点→汇点（设置容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 目标球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            ek.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量为理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)          # 其他球队最多赢到max_win_x场
            ek.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数
    
    # 3. 计算最大流与总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = ek.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 最大流不足则被淘汰
    return is_elim, max_win_x

# 解析5支球队数据集
team_names = ["New_York", "Baltimore", "Boston", "Toronto", "Detroit"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (75, 28),   # New_York：已胜75，剩余28场 → 理论最大103
    (71, 28),   # Baltimore：已胜71，剩余28场 → 理论最大99
    (69, 27),   # Boston：已胜69，剩余27场 → 理论最大96
    (63, 27),   # Toronto：已胜63，剩余27场 → 理论最大90
    (49, 27)    # Detroit：已胜49，剩余27场 → 理论最大76
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [[0 for _ in range(5)] for _ in range(5)]
# 填充上三角矩阵
games[0][1] = 3  # New_York vs Baltimore
games[0][2] = 8  # New_York vs Boston
games[0][3] = 7  # New_York vs Toronto
games[0][4] = 3  # New_York vs Detroit
games[1][2] = 2  # Baltimore vs Boston
games[1][3] = 7  # Baltimore vs Toronto
games[1][4] = 7  # Baltimore vs Detroit
games[2][3] = 0  # Boston vs Toronto
games[2][4] = 3  # Boston vs Detroit
games[3][4] = 3  # Toronto vs Detroit

# 确保矩阵对称性（下三角等于上三角）
for i in range(5):
    for j in range(i+1, 5):
        games[j][i] = games[i][j]

def run_100_times():
    """运行100次并计算平均运行时间"""
    total_time = 0.0
    # 记录每次运行的结果，用于验证一致性
    all_results = []
    
    for run in range(100):
        start_time = time.time()
        
        # 每次运行都重新判断5支球队
        results = []
        for i in range(5):
            is_elim, max_win = is_eliminated(teams, games, i)
            results.append((is_elim, max_win))
        
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time
        all_results.append(results)
        
        # 每10次运行输出一次进度
        if (run + 1) % 10 == 0:
            print(f"完成第{run + 1}次运行，本次耗时: {run_time:.6f}秒")
    
    # 验证所有运行结果是否一致
    first_result = all_results[0]
    consistent = all(result == first_result for result in all_results)
    
    # 计算平均运行时间
    avg_time = total_time / 100
    
    return avg_time, consistent, first_result

# 执行100次运行并获取结果
avg_time, consistent, results = run_100_times()

# 输出最终结果
print("\n--------------------- 运行结果 ---------------------")
for i in range(5):
    is_elim, max_win = results[i]
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")

print(f"\n100次运行的平均时间: {avg_time:.6f}秒")
print(f"所有运行结果是否一致: {'是' if consistent else '否'}")

完成第10次运行，本次耗时: 0.000441秒
完成第20次运行，本次耗时: 0.000427秒
完成第30次运行，本次耗时: 0.000432秒
完成第40次运行，本次耗时: 0.000409秒
完成第50次运行，本次耗时: 0.000404秒
完成第60次运行，本次耗时: 0.000407秒
完成第70次运行，本次耗时: 0.000406秒
完成第80次运行，本次耗时: 0.000407秒
完成第90次运行，本次耗时: 0.000435秒
完成第100次运行，本次耗时: 0.000600秒

--------------------- 运行结果 ---------------------
New_York没有被淘汰。最大的可能获胜场数103
Baltimore没有被淘汰。最大的可能获胜场数99
Boston没有被淘汰。最大的可能获胜场数96
Toronto没有被淘汰。最大的可能获胜场数90
Detroit已经被淘汰了。最大的可能获胜场数76

100次运行的平均时间: 0.000456秒
所有运行结果是否一致: 是


In [10]:
from collections import deque
import time

class EdmondsKarp:
    def __init__(self, n):
        """初始化图结构，n为节点总数"""
        self.n = n
        self.graph = [[0] * n for _ in range(n)]  # 邻接矩阵存储图
    
    def add_edge(self, fr, to, cap):
        """添加边，fr为起点，to为终点，cap为容量"""
        self.graph[fr][to] = cap
    
    def bfs(self, source, sink, parent):
        """BFS寻找从源点到汇点的增广路径，返回是否找到"""
        visited = [False] * self.n
        queue = deque([source])
        visited[source] = True
        parent[:] = [-1] * self.n  # 记录路径
        
        while queue:
            u = queue.popleft()
            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    queue.append(v)
                    parent[v] = u
                    visited[v] = True
                    if v == sink:
                        return True
        return False
    
    def max_flow(self, source, sink):
        """计算从源点到汇点的最大流"""
        parent = [-1] * self.n
        max_flow = 0
        
        while self.bfs(source, sink, parent):
            # 找到增广路径上的最小残留容量
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.graph[u][v])
                v = u
            
            # 更新最大流
            max_flow += path_flow
            
            # 更新残留网络（添加反向边）
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减少流量
                self.graph[v][u] += path_flow  # 反向边增加流量
                v = u
        
        return max_flow

def is_eliminated(teams, games, team_idx):
    """判断球队是否被淘汰，返回(是否淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    ek = EdmondsKarp(total_nodes)
    
    # 1. 源点→比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                ek.add_edge(s, game_idx, games[i][j])                # 源点到比赛节点
                ek.add_edge(game_idx, game_nodes + i + 1, 10**9)     # 比赛到球队i
                ek.add_edge(game_idx, game_nodes + j + 1, 10**9)     # 比赛到球队j
                game_idx += 1
    
    # 2. 球队节点→汇点（设置容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 目标球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            ek.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量为理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)          # 其他球队最多赢到max_win_x场
            ek.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数
    
    # 3. 计算最大流与总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = ek.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 最大流不足则被淘汰
    return is_elim, max_win_x

# 解析12支球队数据集
team_names = ["Poland", "Russia", "Brazil", "Iran", "Italy", "Cuba", "Argentina", "USA", "Japan", "Serbia", "Egypt", "China"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (6, 4),    # Poland：已胜6，剩余4场 → 理论最大10
    (5, 5),    # Russia：已胜5，剩余5场 → 理论最大10
    (5, 5),    # Brazil：已胜5，剩余5场 → 理论最大10
    (5, 4),    # Iran：已胜5，剩余4场 → 理论最大9
    (4, 5),    # Italy：已胜4，剩余5场 → 理论最大9
    (4, 5),    # Cuba：已胜4，剩余5场 → 理论最大9
    (3, 5),    # Argentina：已胜3，剩余5场 → 理论最大8
    (3, 4),    # USA：已胜3，剩余4场 → 理论最大7
    (2, 4),    # Japan：已胜2，剩余4场 → 理论最大6
    (1, 5),    # Serbia：已胜1，剩余5场 → 理论最大6
    (1, 4),    # Egypt：已胜1，剩余4场 → 理论最大5
    (0, 4)     # China：已胜0，剩余4场 → 理论最大4
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [[0 for _ in range(12)] for _ in range(12)]

# 解析每支球队与其他球队的剩余比赛场数
# Poland: 0 1 1 0 1 0 0 0 0 0 1 0 → 与Russia(1)、Brazil(1)、Iran(0)、Italy(1)、Cuba(0)、Argentina(0)、USA(0)、Japan(0)、Serbia(0)、Egypt(1)、China(0)
games[0][1] = 1; games[0][2] = 1; games[0][3] = 0; games[0][4] = 1; games[0][5] = 0
games[0][6] = 0; games[0][7] = 0; games[0][8] = 0; games[0][9] = 0; games[0][10] = 1; games[0][11] = 0

# Russia: 1 0 0 1 0 1 1 0 1 0 0 0 → 与Brazil(0)、Iran(1)、Italy(0)、Cuba(1)、Argentina(1)、USA(0)、Japan(1)、Serbia(0)、Egypt(0)、China(0)
games[1][2] = 0; games[1][3] = 1; games[1][4] = 0; games[1][5] = 1; games[1][6] = 1
games[1][7] = 0; games[1][8] = 1; games[1][9] = 0; games[1][10] = 0; games[1][11] = 0

# Brazil: 1 0 0 1 0 1 0 0 1 1 0 0 → 与Iran(1)、Italy(0)、Cuba(1)、Argentina(0)、USA(0)、Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[2][3] = 1; games[2][4] = 0; games[2][5] = 1; games[2][6] = 0; games[2][7] = 0
games[2][8] = 1; games[2][9] = 1; games[2][10] = 0; games[2][11] = 0

# Iran: 0 1 1 0 1 0 0 0 0 0 0 1 → 与Italy(1)、Cuba(1)、Argentina(0)、USA(0)、Japan(0)、Serbia(0)、Egypt(0)、China(1)
games[3][4] = 1; games[3][5] = 1; games[3][6] = 0; games[3][7] = 0; games[3][8] = 0
games[3][9] = 0; games[3][10] = 0; games[3][11] = 1

# Italy: 1 0 0 1 0 0 1 0 1 1 0 0 → 与Cuba(0)、Argentina(1)、USA(0)、Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[4][5] = 0; games[4][6] = 1; games[4][7] = 0; games[4][8] = 1; games[4][9] = 1
games[4][10] = 0; games[4][11] = 0

# Cuba: 0 1 1 0 0 0 0 1 0 0 1 1 → 与Argentina(0)、USA(1)、Japan(0)、Serbia(0)、Egypt(1)、China(1)
games[5][6] = 0; games[5][7] = 1; games[5][8] = 0; games[5][9] = 0; games[5][10] = 1; games[5][11] = 1

# Argentina: 0 1 0 0 1 0 0 1 0 0 1 1 → 与USA(1)、Japan(0)、Serbia(0)、Egypt(1)、China(1)
games[6][7] = 1; games[6][8] = 0; games[6][9] = 0; games[6][10] = 1; games[6][11] = 1

# USA: 0 0 0 0 0 1 1 0 1 1 0 0 → 与Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[7][8] = 1; games[7][9] = 1; games[7][10] = 0; games[7][11] = 0

# Japan: 0 1 1 0 1 0 0 1 0 0 0 0 → 与Serbia(0)、Egypt(0)、China(0)
games[8][9] = 0; games[8][10] = 0; games[8][11] = 0

# Serbia: 0 0 1 0 1 0 0 1 0 0 1 1 → 与Egypt(1)、China(1)
games[9][10] = 1; games[9][11] = 1

# Egypt: 1 0 0 0 0 1 1 0 0 1 0 0 → 与China(0)
games[10][11] = 0

# 确保矩阵对称性（下三角等于上三角）
for i in range(12):
    for j in range(i+1, 12):
        games[j][i] = games[i][j]

# 主程序：逐个判断球队 + 统计运行时间
start_time = time.time()
for i in range(12):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

Poland没有被淘汰。最大的可能获胜场数10
Russia没有被淘汰。最大的可能获胜场数10
Brazil没有被淘汰。最大的可能获胜场数10
Iran没有被淘汰。最大的可能获胜场数9
Italy没有被淘汰。最大的可能获胜场数9
Cuba没有被淘汰。最大的可能获胜场数9
Argentina没有被淘汰。最大的可能获胜场数8
USA没有被淘汰。最大的可能获胜场数7
Japan已经被淘汰了。最大的可能获胜场数6
Serbia已经被淘汰了。最大的可能获胜场数6
Egypt已经被淘汰了。最大的可能获胜场数5
China已经被淘汰了。最大的可能获胜场数4

总运行时间: 0.018193 秒


In [11]:
from collections import deque
import time

class EdmondsKarp:
    def __init__(self, n):
        """初始化图结构，n为节点总数"""
        self.n = n
        self.graph = [[0] * n for _ in range(n)]  # 邻接矩阵存储图
    
    def add_edge(self, fr, to, cap):
        """添加边，fr为起点，to为终点，cap为容量"""
        self.graph[fr][to] = cap
    
    def bfs(self, source, sink, parent):
        """BFS寻找从源点到汇点的增广路径，返回是否找到"""
        visited = [False] * self.n
        queue = deque([source])
        visited[source] = True
        parent[:] = [-1] * self.n  # 记录路径
        
        while queue:
            u = queue.popleft()
            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    queue.append(v)
                    parent[v] = u
                    visited[v] = True
                    if v == sink:
                        return True
        return False
    
    def max_flow(self, source, sink):
        """计算从源点到汇点的最大流"""
        parent = [-1] * self.n
        max_flow = 0
        
        while self.bfs(source, sink, parent):
            # 找到增广路径上的最小残留容量
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.graph[u][v])
                v = u
            
            # 更新最大流
            max_flow += path_flow
            
            # 更新残留网络（添加反向边）
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减少流量
                self.graph[v][u] += path_flow  # 反向边增加流量
                v = u
        
        return max_flow

def is_eliminated(teams, games, team_idx):
    """判断球队是否被淘汰，返回(是否淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    ek = EdmondsKarp(total_nodes)
    
    # 1. 源点→比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                ek.add_edge(s, game_idx, games[i][j])                # 源点到比赛节点
                ek.add_edge(game_idx, game_nodes + i + 1, 10**9)     # 比赛到球队i
                ek.add_edge(game_idx, game_nodes + j + 1, 10**9)     # 比赛到球队j
                game_idx += 1
    
    # 2. 球队节点→汇点（设置容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 目标球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            ek.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量为理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)          # 其他球队最多赢到max_win_x场
            ek.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数
    
    # 3. 计算最大流与总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = ek.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 最大流不足则被淘汰
    return is_elim, max_win_x

# 解析12支球队数据集
team_names = ["Poland", "Russia", "Brazil", "Iran", "Italy", "Cuba", "Argentina", "USA", "Japan", "Serbia", "Egypt", "China"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (6, 4),    # Poland：已胜6，剩余4场 → 理论最大10
    (5, 5),    # Russia：已胜5，剩余5场 → 理论最大10
    (5, 5),    # Brazil：已胜5，剩余5场 → 理论最大10
    (5, 4),    # Iran：已胜5，剩余4场 → 理论最大9
    (4, 5),    # Italy：已胜4，剩余5场 → 理论最大9
    (4, 5),    # Cuba：已胜4，剩余5场 → 理论最大9
    (3, 5),    # Argentina：已胜3，剩余5场 → 理论最大8
    (3, 4),    # USA：已胜3，剩余4场 → 理论最大7
    (2, 4),    # Japan：已胜2，剩余4场 → 理论最大6
    (1, 5),    # Serbia：已胜1，剩余5场 → 理论最大6
    (1, 4),    # Egypt：已胜1，剩余4场 → 理论最大5
    (0, 4)     # China：已胜0，剩余4场 → 理论最大4
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [[0 for _ in range(12)] for _ in range(12)]

# 解析每支球队与其他球队的剩余比赛场数
# Poland: 0 1 1 0 1 0 0 0 0 0 1 0 → 与Russia(1)、Brazil(1)、Iran(0)、Italy(1)、Cuba(0)、Argentina(0)、USA(0)、Japan(0)、Serbia(0)、Egypt(1)、China(0)
games[0][1] = 1; games[0][2] = 1; games[0][3] = 0; games[0][4] = 1; games[0][5] = 0
games[0][6] = 0; games[0][7] = 0; games[0][8] = 0; games[0][9] = 0; games[0][10] = 1; games[0][11] = 0

# Russia: 1 0 0 1 0 1 1 0 1 0 0 0 → 与Brazil(0)、Iran(1)、Italy(0)、Cuba(1)、Argentina(1)、USA(0)、Japan(1)、Serbia(0)、Egypt(0)、China(0)
games[1][2] = 0; games[1][3] = 1; games[1][4] = 0; games[1][5] = 1; games[1][6] = 1
games[1][7] = 0; games[1][8] = 1; games[1][9] = 0; games[1][10] = 0; games[1][11] = 0

# Brazil: 1 0 0 1 0 1 0 0 1 1 0 0 → 与Iran(1)、Italy(0)、Cuba(1)、Argentina(0)、USA(0)、Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[2][3] = 1; games[2][4] = 0; games[2][5] = 1; games[2][6] = 0; games[2][7] = 0
games[2][8] = 1; games[2][9] = 1; games[2][10] = 0; games[2][11] = 0

# Iran: 0 1 1 0 1 0 0 0 0 0 0 1 → 与Italy(1)、Cuba(1)、Argentina(0)、USA(0)、Japan(0)、Serbia(0)、Egypt(0)、China(1)
games[3][4] = 1; games[3][5] = 1; games[3][6] = 0; games[3][7] = 0; games[3][8] = 0
games[3][9] = 0; games[3][10] = 0; games[3][11] = 1

# Italy: 1 0 0 1 0 0 1 0 1 1 0 0 → 与Cuba(0)、Argentina(1)、USA(0)、Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[4][5] = 0; games[4][6] = 1; games[4][7] = 0; games[4][8] = 1; games[4][9] = 1
games[4][10] = 0; games[4][11] = 0

# Cuba: 0 1 1 0 0 0 0 1 0 0 1 1 → 与Argentina(0)、USA(1)、Japan(0)、Serbia(0)、Egypt(1)、China(1)
games[5][6] = 0; games[5][7] = 1; games[5][8] = 0; games[5][9] = 0; games[5][10] = 1; games[5][11] = 1

# Argentina: 0 1 0 0 1 0 0 1 0 0 1 1 → 与USA(1)、Japan(0)、Serbia(0)、Egypt(1)、China(1)
games[6][7] = 1; games[6][8] = 0; games[6][9] = 0; games[6][10] = 1; games[6][11] = 1

# USA: 0 0 0 0 0 1 1 0 1 1 0 0 → 与Japan(1)、Serbia(1)、Egypt(0)、China(0)
games[7][8] = 1; games[7][9] = 1; games[7][10] = 0; games[7][11] = 0

# Japan: 0 1 1 0 1 0 0 1 0 0 0 0 → 与Serbia(0)、Egypt(0)、China(0)
games[8][9] = 0; games[8][10] = 0; games[8][11] = 0

# Serbia: 0 0 1 0 1 0 0 1 0 0 1 1 → 与Egypt(1)、China(1)
games[9][10] = 1; games[9][11] = 1

# Egypt: 1 0 0 0 0 1 1 0 0 1 0 0 → 与China(0)
games[10][11] = 0

# 确保矩阵对称性（下三角等于上三角）
for i in range(12):
    for j in range(i+1, 12):
        games[j][i] = games[i][j]

def run_100_times():
    """运行100次并计算平均运行时间"""
    total_time = 0.0
    # 记录每次运行的结果，用于验证一致性
    all_results = []
    
    for run in range(100):
        start_time = time.time()
        
        # 每次运行都重新判断12支球队
        results = []
        for i in range(12):
            is_elim, max_win = is_eliminated(teams, games, i)
            results.append((is_elim, max_win))
        
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time
        all_results.append(results)
        
        # 每10次运行输出一次进度
        if (run + 1) % 10 == 0:
            print(f"完成第{run + 1}次运行，本次耗时: {run_time:.6f}秒")
    
    # 验证所有运行结果是否一致
    first_result = all_results[0]
    consistent = all(result == first_result for result in all_results)
    
    # 计算平均运行时间
    avg_time = total_time / 100
    
    return avg_time, consistent, first_result

# 执行100次运行并获取结果
avg_time, consistent, results = run_100_times()

# 输出最终结果
print("\n--------------------- 运行结果 ---------------------")
for i in range(12):
    is_elim, max_win = results[i]
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")

print(f"\n100次运行的平均时间: {avg_time:.6f}秒")
print(f"所有运行结果是否一致: {'是' if consistent else '否'}")

完成第10次运行，本次耗时: 0.016278秒
完成第20次运行，本次耗时: 0.019508秒
完成第30次运行，本次耗时: 0.016967秒
完成第40次运行，本次耗时: 0.018363秒
完成第50次运行，本次耗时: 0.017075秒
完成第60次运行，本次耗时: 0.017763秒
完成第70次运行，本次耗时: 0.016861秒
完成第80次运行，本次耗时: 0.023299秒
完成第90次运行，本次耗时: 0.017714秒
完成第100次运行，本次耗时: 0.018013秒

--------------------- 运行结果 ---------------------
Poland没有被淘汰。最大的可能获胜场数10
Russia没有被淘汰。最大的可能获胜场数10
Brazil没有被淘汰。最大的可能获胜场数10
Iran没有被淘汰。最大的可能获胜场数9
Italy没有被淘汰。最大的可能获胜场数9
Cuba没有被淘汰。最大的可能获胜场数9
Argentina没有被淘汰。最大的可能获胜场数8
USA没有被淘汰。最大的可能获胜场数7
Japan已经被淘汰了。最大的可能获胜场数6
Serbia已经被淘汰了。最大的可能获胜场数6
Egypt已经被淘汰了。最大的可能获胜场数5
China已经被淘汰了。最大的可能获胜场数4

100次运行的平均时间: 0.018314秒
所有运行结果是否一致: 是
